In [1]:
from stable_baselines import PPO2, TRPO, DDPG, A2C
import seagul.envs
import gym

env_name = 'su_acrocot-v0'

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv

import numpy as np
import matplotlib.pyplot as plt

import os
import gym
import numpy as np
import matplotlib.pyplot as plt

from stable_baselines.ddpg.policies import LnMlpPolicy
from stable_baselines.bench import Monitor
from stable_baselines.results_plotter import load_results, ts2xy
from stable_baselines import DDPG
from stable_baselines.ddpg import AdaptiveParamNoiseSpec

best_mean_reward, n_steps = -np.inf, 0
env = DummyVecEnv([lambda: env])  # The algorithms require a vectorized environment to run

models = []
model = PPO2('MlpPolicy', env, 
             #nb_rollout_steps=500,
             #normalize_observations=True,
             #batch_size = 512,
             verbose=False,
            )
model.learn(1228800, seed=1)


ModuleNotFoundError: No module named 'tensorflow.contrib'

In [13]:
myenv = env
myenv.num_steps=1000

#model.gate_fn.net_fn = gate
action_hist = np.zeros((myenv.num_steps,1))
state_hist = np.zeros((myenv.num_steps, myenv.observation_space.shape[0]))
reward_hist = np.zeros((myenv.num_steps, 1))

obs = env.reset()

num_trials = 100
rewards = np.zeros((num_trials))

for trial in range(num_trials):
    obs = env.reset()
    for i in range(myenv.num_steps):
            actions, _states, = model.predict(obs)
            #actions = np.ones(1)*100
            obs, reward, done, _ = env.step(actions)        
            action_hist[i,:] = np.copy(actions)
            state_hist[i,:] = np.copy(obs)
            reward_hist[i,:] = np.copy(reward)
            env.render()
            if done:
                break

    rewards[trial] = sum(reward_hist)

            
        

# plt.plot(action_hist)
# plt.title('actions')

# plt.figure()
# plt.plot(reward_hist)
# plt.title('reward')

# plt.figure()
# plt.plot(state_hist[:,0])
# plt.title("theta")

# plt.figure()
# plt.plot(state_hist[:,1])
# plt.title("x")


# plt.figure()
# plt.plot(state_hist[:,2])
# plt.title("theta dot")


# plt.figure()
# plt.plot(state_hist[:,3])
# plt.title("xdot ")



print(i)
#print(sum(reward_hist))
print(rewards.mean())
print(rewards.std())



999
-129.4850025173438
174.85784558475308


In [ ]:
from numpy import pi

def control(env, q):
    k1 = 140.560
    k2 = -3.162
    k3 = 41.772
    k4 = 0#-8.314
    u = -4*(k1 * (q[0] - pi) + k2 * q[1] + k3 * q[2] + k4 * q[3])
    return u

myenv = env
myenv.num_steps=400

#model.gate_fn.net_fn = gate
action_hist = np.zeros((myenv.num_steps,1))
state_hist = np.zeros((myenv.num_steps, myenv.observation_space.shape[0]))
reward_hist = np.zeros((myenv.num_steps, 1))
path_hist = np.zeros((myenv.num_steps,1))

obs = env.reset()
#env.state[0] = np.pi
model.hyst_state = 0

for i in range(myenv.num_steps):
        if ((155 * pi/180 < obs[0][0] < 220 * pi/180)):
            actions = np.array(control(env, obs[0]))/50
            #actions = np.zeros(1)
            path_hist[i]  = 1
        
        else:
            actions, _,  = model.predict(obs)
            path_hist[i] = 0
            
        
        actions = np.clip(actions, -75, 75)
        #actions = np.zeros(1)
        obs, reward, done, _ = env.step(np.array(actions).reshape(-1,1))
        action_hist[i,:] = np.copy(actions)
        state_hist[i,:] = np.copy(obs)
        reward_hist[i,:] = np.copy(reward)
        #env.render()
        if done:
            break

plt.plot(action_hist)
plt.title('action')


plt.figure()
plt.plot(path_hist)

plt.figure()
plt.plot(state_hist[:,0])
plt.title("theta")

plt.figure()
plt.plot(state_hist[:,1])
plt.title("x")


plt.figure()
plt.plot(state_hist[:,2])
plt.title("theta dot")


plt.figure()
plt.plot(state_hist[:,3])
plt.title("xdot ")

print(i)
print(sum(reward_hist))

In [2]:
rewards

array([[-1967.46878566,     0.        ,     0.        ,     0.        ],
       [-1975.04432526,     0.        ,     0.        ,     0.        ],
       [-1960.63329954,     0.        ,     0.        ,     0.        ],
       [-1959.69436482,     0.        ,     0.        ,     0.        ],
       [-1952.9140735 ,     0.        ,     0.        ,     0.        ],
       [-1971.14081908,     0.        ,     0.        ,     0.        ],
       [-1952.04835808,     0.        ,     0.        ,     0.        ],
       [-1893.88650594,     0.        ,     0.        ,     0.        ],
       [-1966.49728884,     0.        ,     0.        ,     0.        ],
       [-1949.22009965,     0.        ,     0.        ,     0.        ]])